In [1]:
import pandas as pd
# import polars as pl
import sqlite3

Cleaning strategy for NPPES file:
- read in only columns relevant to this project
- process taxonomy code columns to determine primary
- drop taxonomy code columns (keep only added primary code column)
- join classification crosswalk to get taxonomy name
- join CBSA crosswalk to match provider on zip code
- make column names better for SQL
- put into sqlite

In [2]:
# generate list of taxonomy columns
tax_cols = []

for i in range(1, 16):
    switch_col_name = f'Healthcare Provider Primary Taxonomy Switch_{i}'
    tax_cols.append(switch_col_name)
    code_col_name = f'Healthcare Provider Taxonomy Code_{i}'
    tax_cols.append(code_col_name)

# other relevant columns
cols = ['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)',
        'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text',
        'Provider Credential Text', 'Provider First Line Business Practice Location Address',
        'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code']

# combine column name lists
cols.extend(tax_cols)

In [3]:
npi = pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols = cols, dtype = str)

In [4]:
# pull primary taxonomy codes into one column
# modify to take first 'X' if there is no 'Y'
# add unified flag column (X or Y)
# pull all rows with 'X' into separate dataset (in addition to main)
# don't drop non-flag

def extract_taxonomy_code(row):

#    row['extracted_switch_col'] = None

    # to catch all rows with a 'Y' switch
    for i in range(1, 16):
        switch_col_name = f'Healthcare Provider Primary Taxonomy Switch_{i}'
        if row[switch_col_name] == 'Y':
            code_col_name = f'Healthcare Provider Taxonomy Code_{i}'
            # row['extracted_switch_col'] = 'Y'
            return row[code_col_name]
        
    # if no 'Y' switch was found, capture first 'X'
    # this second chunk makes it incredibly inefficient, find a better way
    # if row['extracted_switch_col'] is None:
    #     for i in range(1, 16):
    #         switch_col_name = f'Healthcare Provider Primary Taxonomy Switch_{i}'
    #         if row[switch_col_name] == 'X':
    #             code_col_name = f'Healthcare Provider Taxonomy Code_{i}'
    #             if 'extracted_switch_col' not in row:
    #                 row['extracted_switch_col'] = 'X'
    #                 return row[code_col_name]
    
    return None # for rows with no 'Y' or 'X' switch

In [5]:
npi['extracted_taxonomy_code'] = npi.apply(extract_taxonomy_code, axis = 1)

KeyboardInterrupt: 

In [ ]:
npi.head()

In [ ]:
npi.info()
# having doublechecked the values in the switch columns, it appears that there are rows with no 'Y' switch
# should those rows be dropped, or possibly the taxonomy code in Code_1 could be inferred to be the correct code?
# some rows with no 'Y' switch have one or more 'X' switches

In [ ]:
npi['Provider Business Practice Location Address Postal Code'].value_counts(dropna = False)

In [ ]:
# truncate zipcode to 5 digits (match CBSA crosswalk)
npi['Provider Business Practice Location Address Postal Code'] = npi['Provider Business Practice Location Address Postal Code'].str.slice(0, 5)

In [ ]:
# if we decide to drop rows with no 'Y' switch
# npi.dropna(subset = ['extracted_taxonomy_code'], inplace = True)

In [ ]:
# drop taxonomy columns
npi = npi.drop(tax_cols, axis = 1)

In [ ]:
# convert column names to better format
npi.columns = [x.lower().replace(' ', '_').replace('(', '').replace(')', '') for x in npi.columns]

In [ ]:
# npi data to sqlite
# create sqlite database
db = sqlite3.connect('../data/provider.sqlite')

npi.to_sql('npi',
           db,
           if_exists = 'replace',
           index = False)

db.close()

In [8]:
#create indexes on npi-containing columns for faster queries
db = sqlite3.connect('../data/provider.sqlite')

db.execute('CREATE INDEX npi_index ON npi(npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')
db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.close()

In [ ]:
# read in classification crosswalk
class_cross = pd.read_csv('../data/nucc_taxonomy_240.csv', usecols = ['Code', 'Grouping', 'Classification', 'Display Name'])

In [ ]:
class_cross.head()

In [ ]:
# convert column names to better format
class_cross.columns = [x.lower().replace(' ', '_') for x in class_cross.columns]

In [ ]:
# classification crosswalk data to sqlite
# create sqlite database
db = sqlite3.connect('../data/provider.sqlite')

class_cross.to_sql('class_cross',
           db,
           if_exists = 'replace',
           index = False)

db.close()

In [ ]:
# read in CBSA crosswalk

cbsa_cross = pd.read_csv('../data/ZIP_CBSA_122023.xlsx - Export Worksheet.csv', usecols = ['ZIP', 'CBSA', 'USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE'])

In [ ]:
cbsa_cross.head()

In [ ]:
cbsa_cross.info()

In [ ]:
# convert column names to better format
cbsa_cross.columns = [x.lower().replace(' ', '_') for x in cbsa_cross.columns]

In [ ]:
# CBSA crosswalk data to sqlite
# create sqlite database
db = sqlite3.connect('../data/provider.sqlite')

cbsa_cross.to_sql('cbsa_cross',
           db,
           if_exists = 'replace',
           index = False)

db.close()

In [ ]:
# read in hop teaming data
hop = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv')

In [ ]:
hop.head()

In [ ]:
hop.info()

In [ ]:
hop = hop[hop['transaction_count'] >= 50]

In [ ]:
hop = hop[hop['average_day_wait'] <= 50]

In [ ]:
hop.head()

In [ ]:
# hop data to sqlite
# create sqlite database
db = sqlite3.connect('../data/provider.sqlite')

hop.to_sql('hop',
           db,
           if_exists = 'replace',
           index = False)

db.close()

In [6]:
# building table for hospitals
# cbsa = 34980, either hardcode or use a cte to get the value where usps_zip_pref_city = "NASHVILLE" and usps_zip_pref_state = "TN" (filter based on referee cbsa)
query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_sqlite = pd.read_sql(query, db)

hosp_sqlite.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1881697092,2,"RHS, INC.",1330 TROTWOOD AVE,None,COLUMBIA,TN,38401,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
1,1326041534,2,MURFREESBORO CONVALESCENT SERVICE,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,37130,341600000X,Transportation Services,Ambulance,Ambulance
2,1063415834,2,ADVANCED DIABETIC SERVICES,151 HERITAGE PARK DR,STE 403,MURFREESBORO,TN,37129,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
3,1538162334,2,TENNESSEE IMAGING ALLIANCE LLC,2628 N MOUNT JULIET RD,None,MOUNT JULIET,TN,37122,261QR0208X,Ambulatory Health Care Facilities,Clinic/Center,Mobile Radiology Clinic/Center
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center


In [54]:
#overview of entity categories
unique_code_group = hosp_sqlite.code_group.unique()

print(unique_code_group)

['Suppliers' 'Transportation Services' 'Ambulatory Health Care Facilities'
 'Allopathic & Osteopathic Physicians' 'Agencies' None
 'Other Service Providers' 'Laboratories'
 'Nursing & Custodial Care Facilities'
 'Respiratory, Developmental, Rehabilitative and Restorative Service Providers'
 'Hospitals' 'Residential Treatment Facilities'
 'Eye and Vision Services Providers' 'Dental Providers'
 'Pharmacy Service Providers' 'Hospital Units'
 'Physician Assistants & Advanced Practice Nursing Providers'
 'Behavioral Health & Social Service Providers'
 'Podiatric Medicine & Surgery Service Providers'
 'Technologists, Technicians & Other Technical Service Providers'
 'Chiropractic Providers' 'Nursing Service Providers'
 'Speech, Language and Hearing Service Providers'
 'Managed Care Organizations' 'Emergency Medical Service Providers'
 'Nursing Service Related Providers'
 'Dietary & Nutritional Service Providers' 'Respite Care Facility'
 'Student, Health Care']


In [9]:
#left joining hop data

query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name,
    from_npi as referred_from,
    patient_count,
    transaction_count,
    average_day_wait,
    std_day_wait
    
    
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
LEFT JOIN hop
    ON n.npi = hop.to_npi
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_hop = pd.read_sql(query, db)

hosp_hop.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name,referred_from,patient_count,transaction_count,average_day_wait,std_day_wait
0,1881697092,2,"RHS, INC.",1330 TROTWOOD AVE,None,COLUMBIA,TN,38401,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies,NaN,NaN,NaN,NaN,NaN
1,1326041534,2,MURFREESBORO CONVALESCENT SERVICE,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,37130,341600000X,Transportation Services,Ambulance,Ambulance,NaN,NaN,NaN,NaN,NaN
2,1063415834,2,ADVANCED DIABETIC SERVICES,151 HERITAGE PARK DR,STE 403,MURFREESBORO,TN,37129,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies,NaN,NaN,NaN,NaN,NaN
3,1538162334,2,TENNESSEE IMAGING ALLIANCE LLC,2628 N MOUNT JULIET RD,None,MOUNT JULIET,TN,37122,261QR0208X,Ambulatory Health Care Facilities,Clinic/Center,Mobile Radiology Clinic/Center,NaN,NaN,NaN,NaN,NaN
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.518961e+09,264.0,451.0,0.929,10.235


In [64]:
hosp_sqlite.shape

(9841, 12)

In [61]:
hosp_hop.shape

(131530, 17)

In [ ]:
#removing NaNs fixed issue of referred_from showing up in scientific notation

In [26]:
#tried to remove NaNs but actually I think I just need to right join instead. No, INNER JOIN
hosp_hop_nonan = hosp_hop[~hosp_hop['referred_from'].isna()]
hosp_hop_nonan

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name,referred_from,patient_count,transaction_count,average_day_wait,std_day_wait
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.518961e+09,264.0,451.0,0.929,10.235
5,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.538154e+09,89.0,92.0,47.554,81.779
6,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.801075e+09,169.0,193.0,0.000,0.000
7,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.427099e+09,122.0,133.0,0.000,0.000
8,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1.336416e+09,74.0,76.0,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128771,1225517014,2,HOSPITALIST MEDICINE PHYSICIANS OF OHIO - COLU...,5410 MARYLAND WAY STE 300,None,BRENTWOOD,TN,37027,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,Internal Medicine Physician,1.386930e+09,85.0,89.0,5.303,13.041
128772,1225517014,2,HOSPITALIST MEDICINE PHYSICIANS OF OHIO - COLU...,5410 MARYLAND WAY STE 300,None,BRENTWOOD,TN,37027,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,Internal Medicine Physician,1.396836e+09,110.0,218.0,28.610,60.571
128773,1225517014,2,HOSPITALIST MEDICINE PHYSICIANS OF OHIO - COLU...,5410 MARYLAND WAY STE 300,None,BRENTWOOD,TN,37027,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,Internal Medicine Physician,1.396855e+09,42.0,55.0,38.273,76.692
128785,1417438706,2,BECKWITH EMERGENCY PHYSICIAN PLLC,5052 BECKWITH RD,None,MT JULIET,TN,37122,207P00000X,Allopathic & Osteopathic Physicians,Emergency Medicine,Emergency Medicine Physician,1.861478e+09,48.0,50.0,11.500,36.757


In [12]:
#INNER joining hop data
query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name,
    from_npi as referred_from,
    patient_count,
    transaction_count,
    average_day_wait,
    std_day_wait
    
    
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
INNER JOIN hop
    ON n.npi = hop.to_npi
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_hop_right = pd.read_sql(query, db)

hosp_hop_right.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name,referred_from,patient_count,transaction_count,average_day_wait,std_day_wait
0,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1518961176,264,451,0.929,10.235
1,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1538153937,89,92,47.554,81.779
2,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1801075247,169,193,0.000,0.000
3,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1427098912,122,133,0.000,0.000
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center,1336415819,74,76,0.000,0.000


In [15]:
hosp_hop_right.shape

(123310, 17)

In [41]:
#filter data by higher transaction count with same avg day wait to illuminate larger facilities(receiving more referrals)
hop_filtered = hosp_hop_right[(hosp_hop_right['transaction_count'] >= 1500) & (hosp_hop_right['average_day_wait'] <= 50)]
hop_filtered.shape

(3252, 17)

In [49]:
#how many unique to: npis?
hop_filtered.npi.nunique()

366

In [50]:
#how many unique org names?
hop_filtered.provider_org_name.nunique()

315

In [53]:
#how many unique from: npis? this explains discrepancy in row between hop_filtered and louvain
hop_filtered.referred_from.nunique()

1361

In [43]:
#export csv to put into Neo4j
hop_filtered.to_csv('../data/hop_filtered.csv', index= False)

In [45]:
#read in louvain output from Neo4j
louvain = pd.read_csv('../data/hop_filtered_louvain.csv', dtype = str)
louvain.head()

,name,communityId
0,"""1003028770""",60
1,"""1003059247""",759
2,"""1003230236""",105
3,"""1003283201""",362
4,"""1003819277""",105


In [48]:
#strip quotes to prepare for join
louvain['name'] = louvain['name'].str.strip('\"')
louvain

,name,communityId
0,1003028770,60
1,1003059247,759
2,1003230236,105
3,1003283201,362
4,1003819277,105
...,...,...
1499,1992756779,46
1500,1992776405,25
1501,1992893416,28
1502,1992897326,335


In [57]:
# inner merge hosp_hop_right onto louvain
# louvain_info = pd.merge(louvain, hosp_hop_right, left_on= 'name', right_on= 'npi', how= 'inner')
# louvain_info.head()

,name,communityId,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name,referred_from,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003028770,60,1003028770,2,SPRING HILL DERMATOLOGY PLC,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician,1518143122,29,50,32.280,57.153
1,1003028770,60,1003028770,2,SPRING HILL DERMATOLOGY PLC,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician,1538234935,42,66,38.439,54.733
2,1003028770,60,1003028770,2,SPRING HILL DERMATOLOGY PLC,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician,1790170389,43,54,37.537,45.100
3,1003028770,60,1003028770,2,SPRING HILL DERMATOLOGY PLC,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician,1801990049,98,130,48.631,59.293
4,1003028770,60,1003028770,2,SPRING HILL DERMATOLOGY PLC,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician,1811955917,79,96,48.281,52.218


In [58]:
#hmm, why am I getting so many rows? should be 1504 right? same as louvain
# louvain_info.shape

(103391, 19)

In [59]:
#change column name to join instead of merging and creating a million columns
louvain = louvain.rename(columns={'name': 'npi'})
louvain

,npi,communityId
0,1003028770,60
1,1003059247,759
2,1003230236,105
3,1003283201,362
4,1003819277,105
...,...,...
1499,1992756779,46
1500,1992776405,25
1501,1992893416,28
1502,1992897326,335


In [64]:
#left join louvain to hosp_hop_right
louvain_info = louvain.join(hosp_hop_right, on= "npi", how= 'left')
louvain_info

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [62]:
hosp_hop_right.dtypes

npi                     object
entity_type             object
provider_org_name       object
provider_address_1      object
provider_address_2      object
provider_city           object
provider_state          object
provider_post_code      object
tax_code                object
code_group              object
code_classification     object
code_display_name       object
referred_from            int64
patient_count            int64
transaction_count        int64
average_day_wait       float64
std_day_wait           float64
dtype: object

In [63]:
louvain.dtypes

npi            object
communityId    object
dtype: object

In [ ]:
#what gives? my key columns are both objects. investigate tomorrow